In [1]:
!pip install -q timm albumentations

     |████████████████████████████████| 549 kB 36.8 MB/s 
     |████████████████████████████████| 182 kB 74.0 MB/s 


In [2]:
!pip install catalyst

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 31.8 MB/s 
     |████████████████████████████████| 125 kB 65.8 MB/s 
     |████████████████████████████████| 191 kB 3.5 MB/s 


In [3]:
from glob import glob
from sklearn.model_selection import GroupKFold
import torch
from torch import nn
import torch.nn.functional as F
import os
import time
from datetime import datetime
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler
import timm
import shutil
from catalyst.data.sampler import BalanceClassSampler
import warnings

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
#압축파일 그대로 올리는 경우 해당 경로에 압축해제
!unzip -qq '/gdrive/My Drive/image-anomaly-detection.zip'
args = {
    'model_name': 'tf_efficientnet_b0_ns',  # 신경망 구조
    'lr': 1e-3,  # 학습률
    'weight_decay': 0,  # 가중치 감쇠
    'drop_rate': 0.2,  # 학습 시 dropout 비율
    'image_size': 256,  # 이미지 크기
    'num_epochs': 50,  # 학습 반복수
    'batch_size': 16,  # 미니배치 크기
    'num_classes': 2,  # 판별할 클래스 개수
    'num_folds': 5,  # 데이터셋 분할 fold 개수
    'val_fold': 0,  # 검증용 fold 선택
    'seed': 42,  # 랜덤 seed 설정
    'log_step': 50,  # log 남길 iteration 반복 수
    'model_save_step': 5,  # model 저장할 epoch 반복 수
    'workspace_path': '/content',  # 작업 위치
    'checkpoint_dir': './checkpoints',  # 모델 저장 디렉토리
    'pretrained_name': 'tf_efficientnet_b0_ns_model_best.pth',  # 학습한 모델 파일이름 (.pth까지 붙이기)
}

TRAIN_DATA_ROOT_PATH = os.path.join(args['workspace_path'], 'train')
TEST_DATA_ROOT_PATH = os.path.join(args['workspace_path'], 'test')

!mkdir './checkpoints'
def get_net():
    net = timm.create_model(args['model_name'], num_classes=args['num_classes'], pretrained=True)

    return net

def load_checkpoint(model, optimizer, pretrained_path, device):
    state = torch.load(pretrained_path, map_location=device)
    model.load_state_dict(state['model'])
    best_acc = state['best_acc']
    epoch = state['epoch']
    print(f'\t## loaded trained models (epoch: {epoch})\n')
    return model, optimizer, best_acc, epoch


model = get_net().to(device)

optimizer = torch.optim.AdamW(model.parameters(),
                              lr=args['lr'], betas=(0.9, 0.999),
                              weight_decay=args['weight_decay'])

def get_valid_transforms():
    return A.Compose([
            ToTensorV2(p=1.0),
        ], p=1.0)
        

Mounted at /gdrive


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b0_ns-c0e6a31c.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b0_ns-c0e6a31c.pth


# Inference

In [4]:
pretrained_path = os.path.join(args['pretrained_name'])
model, _, _, _ = load_checkpoint(model, optimizer, pretrained_path, device)
model.eval()

	## loaded trained models (epoch: 48)



EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          16, eps=

In [5]:
class DatasetSubmissionRetriever(Dataset):
    def __init__(self, image_names, transforms=None):
        super().__init__()
        self.image_names = image_names
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_name = self.image_names[index]
        image_path = f'{TEST_DATA_ROOT_PATH}/{image_name}'
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image[:,:,1]=0
        image[:,:,2]=0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']

        return image_name, image

    def __len__(self) -> int:
        return self.image_names.shape[0]

In [6]:
dataset = DatasetSubmissionRetriever(
    image_names=np.array([path.replace('\\', '/').split('/')[-1] for path in glob('./test/*.png')]),
    transforms=get_valid_transforms(),
)


test_loader = DataLoader(
    dataset,
    batch_size=args['batch_size'],
    shuffle=False,
    drop_last=False,
)

In [7]:
df_result = pd.DataFrame(columns=['id', 'label'])
for step, (image_names, images) in enumerate(test_loader):
    print(step, end='\r')
    images = images.to(device)
    output = model(images)
    class_prob = F.softmax(output, dim=1)
    _, class_pred = output.topk(1, 1, True, True)  # prediction: select k maximum at each output
    label = class_pred.view(-1).detach().cpu().numpy()
    df_curr = pd.DataFrame({
        'id': image_names,
        'label': label
    })
    df_result = pd.concat([df_result, df_curr], axis=0, ignore_index=True)

In [8]:
current_time = datetime.now().strftime(r'%y-%m-%d_%H-%M-%S')
df_result.to_csv(f'submission_{current_time}.csv', index=False)
df_result.head()

,id,label
0,11068.png,1
1,10819.png,0
2,11732.png,0
3,10325.png,0
4,11849.png,1
